In [1]:
import pandas as pd
import nibabel as nb
import os

In [2]:
df = pd.read_csv('/home/abhijit/Jyotirmay/my_thesis/projects/MC_dropout_quicknat/reports/MC_dropout_quicknat_UKB_v2/UKB/10_0.0_report.csv', index_col=0)

In [4]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,17900,17901,17902,17903,17904,17905,17906,17907,17908,17909
project,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
dataset,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
model_name,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,...,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2,MC_dropout_quicknat_UKB_v2
volume_id,1000410_20201_2_0,1000698_20201_2_0,1000857_20201_2_0,1000870_20201_2_0,1000870_20201_2_0,1002272_20201_2_0,1002272_20201_2_0,1002359_20201_2_0,1002876_20201_2_0,1003082_20201_2_0,...,4976677_20201_2_0,4977099_20201_2_0,4977950_20201_2_0,4977996_20201_2_0,4977996_20201_2_0,4978218_20201_2_0,4978325_20201_2_0,4978463_20201_2_0,4978640_20201_2_0,4978731_20201_2_0
samples,10,10,10,10,10,10,10,10,10,10,...,10,10,10,10,10,10,10,10,10,10
sncc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ged,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iou_spleen,0.545999,0.564894,0.7192,0.738249,0.335961,0.436899,0.875844,0.84952,0.776984,0.418048,...,0.82513,0.786793,0.811625,0.688142,0.48167,0.725161,0.83553,0.740236,0.698465,0.508671
iou_liver,0.859389,0.766553,0.851086,0.585848,0.356094,0.369021,0.705661,0.884133,0.819199,0.798031,...,0.877845,0.755698,0.791493,0.373203,0.740073,0.854859,0.846397,0.842993,0.816107,0.329005
iou_mean,0.702694,0.665723,0.785143,0.662049,0.346028,0.40296,0.790752,0.866826,0.798091,0.60804,...,0.851487,0.771246,0.801559,0.530672,0.610871,0.79001,0.840963,0.791615,0.757286,0.418838


In [3]:
with open('/home/abhijit/Jyotirmay/my_thesis/dataset_groups/whole_body_datasets/UKB/evaluated_vols.txt') as file_handle:
        volumes_to_use = file_handle.read().splitlines()

In [19]:
f_paths = []
for ix, v in enumerate(volumes_to_use):
    targets = df[df['volume_id']==v]['target_scan_file'].values
    i1, i2 = None, None
    if len(targets)>1:
        i1, i2 = df[df['volume_id']==v]['iou_mean'].values
        if i1>i2:
            idx = 0
        else:
            idx = 1
        target = targets[idx]
    elif len(targets)==0:
        continue
    else:
        target = targets[0]
    input_file = '/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/'+str(v)+'/'+target
    gt_file = None
    f_paths.append([input_file, gt_file])


['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['

['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'D

['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']

[]
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
[]
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']


[]
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
[]
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.ni

...

['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon

['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['

['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']

['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
[]
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']


['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz'
 'Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']
['Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz']

In [21]:
len(f_paths)

8566

In [16]:
f_paths

[[array(['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1000410_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'],
        dtype=object), None],
 [array(['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1000698_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'],
        dtype=object),
  None],
 [array(['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1000857_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'],
        dtype=object), None],
 ['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1000870_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17s.nii.gz',
  None],
 ['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1002272_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sa.nii.gz',
  None],
 [array(['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nifti/1002359_20201_2_0/Dixon_BH_17s_opp_Dixon_BH_17sb.nii.gz'],
        dtype=object), None],
 [array(['/home/abhijit/nas_drive/Data_WholeBody/UKBiobank/body/body_nift